In [ ]:
import os

import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
from dotenv import load_dotenv


In [49]:
load_dotenv()

True

In [50]:
bucket = "sensor"
org = "hack"
token = "hbxyqRw8XaDPbQb9az1RuZowZvgYBf_FO90BYiRz5PciaPbPIPYFgHZSG_KrVmfw7sdVq7p1R0DQT1yHwdGYkA=="
url="http://localhost:8086"
client = influxdb_client.InfluxDBClient(
   url=url,
   token=token,
   org=org
)
write_api = client.write_api(write_options=SYNCHRONOUS)

In [52]:
FILE_DIR = "hackathon_air_meteo_noise_csv"
METEO = "hackathon_meteo_"
NOISE = "hackathon_noise_"
AIR = "hackathon_air_"

In [69]:
raw_meteodata = []
raw_noisedata = []
raw_airdata = []

for file in os.walk(FILE_DIR):
    if file[0] == FILE_DIR:
        for f in file[2]:
            if f.startswith(METEO):
                raw_meteodata.append(pd.read_csv(os.path.join(FILE_DIR, f), sep=";"))
            elif f.startswith(NOISE):
                raw_noisedata.append(pd.read_csv(os.path.join(FILE_DIR, f), sep=";"))
            elif f.startswith(AIR):
                raw_airdata.append(pd.read_csv(os.path.join(FILE_DIR, f), sep=";"))

In [70]:
col_units_map = {}
for lst in [raw_meteodata, raw_noisedata, raw_airdata]:
    for df in lst:
        # Remove units
        for col in df.columns:
            if " (" in col:
                col_units_map[col.split(" (")[0]] = col.split(" (")[1][:-1]
        df.columns = [col.split(" (")[0] for col in df.columns]
        # Convert types
        df["device name"] = df["device name"].astype("string")
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        # Replace NaN with first value
        df["device name"].ffill(inplace=True)
        df["latitude"].ffill(inplace=True)
        df["longitude"].ffill(inplace=True)

In [71]:
for df in raw_airdata:
    df.set_index("timestamp", inplace=True)
    write_api.write(bucket=bucket, org=org, record=df, data_frame_measurement_name="air", data_frame_tag_columns=["device name", "latitude", "longitude"])
for df in raw_noisedata:
    df.set_index("timestamp", inplace=True)
    write_api.write(bucket=bucket, org=org, record=df, data_frame_measurement_name="noise", data_frame_tag_columns=["device name", "latitude", "longitude"])
for df in raw_meteodata:
    df.set_index("timestamp", inplace=True)
    write_api.write(bucket=bucket, org=org, record=df, data_frame_measurement_name="meteo", data_frame_tag_columns=["device name", "latitude", "longitude"])